# Kredi Kartı Dolandırıcılığı Tespiti

Önceki egzersizler, bir sinir ağının tüm farklı bileşenlerine daha yakından bakmanızı sağladı:
* sıralı (sequential) yoğun (Dense) bir Sinir Ağı mimarisi,
* derleme (compilation) yöntemi,
* modelin eğitilmesi (fitting).

Şimdi **çok fazla veri içeren** gerçek hayat bir veri seti üzerinde çalışalım!

**Veri seti: `Kredi Kartı İşlemleri (Credit Card Transactions)`**

Bu açık uçlu challenge’da, **kredi kartı işlemlerinden elde edilmiş verilerle** çalışacaksınız.

Bu veriler **hassas** olduğu için, toplam 31 sütundan yalnızca 3’ü bilinmektedir; geri kalanlar verileri **anonimleştirmek** amacıyla dönüştürülmüştür (aslında bunlar, orijinal verilerin **PCA projeksiyonlarıdır**).

Bilinen 3 sütun şunlardır:

* `TIME`: İşlemin, veri setindeki ilk işleme göre geçen süresi  
* `AMOUNT`: İşlem tutarı  
* `CLASS` (hedef değişkenimiz):
    * `0 : geçerli işlem`
    * `1 : sahte (fraud) işlem`

❓ **Soru** ❓ Veri setini indirerek başlayın:
* Kaggle üzerinden: [buradan](https://www.kaggle.com/mlg-ulb/creditcardfraud)
* veya bizim URL’mizden: [buradan](https://d32aokrjazspmn.cloudfront.net/materials/creditcard.csv)

Veriyi yükleyerek `X` ve `y` değişkenlerini oluşturun.

In [15]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report,roc_auc_score
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras import layers, models,regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score

df = pd.read_csv("creditcard.csv")
df.head()
X = df.drop(columns = ["Class"])
y = df["Class"]

## 1. Sınıfların yeniden dengelenmesi

In [2]:
# Sınıf dengesini kontrol edelim
pd.Series(y).value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64

☝️ Bu `fraud detection` (sahtekârlık tespiti) challenge’ında **sınıflar aşırı derecede dengesizdir**:
* %99.8 normal işlemler
* %0.2 sahte (fraud) işlemler

**Ciddi yeniden dengeleme (rebalancing) stratejileri uygulamazsak sahtekârlık vakalarını tespit edemeyiz!**

❓ **Soru** ❓
1. **Önce**, veri setinizden üç ayrı bölme oluşturun: `Train / Val / Test`.  
   Doğrulama (validation) ve test setlerinin **dengesiz** kalması son derece önemlidir; böylece modeli değerlendirirken gerçek koşullar korunur ve veri sızıntısı (data leakage) oluşmaz. Test setinizi bu notebook’un **en son hücresine kadar saklayın**!

&nbsp;

2. **İkinci olarak**, yalnızca eğitim setinizi (training set) yeniden dengeleyin. Birçok seçeneğiniz var:

- Azınlık sınıfını rastgele oversample etmek (düz NumPy fonksiyonlarıyla).  
  *(En iyi seçenek değildir; çünkü satırları kopyalayarak veri sızıntısı yaratır.)*
- <a href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">**`Synthetic Minority Oversampling Technique - SMOTE`**</a> kullanarak, mevcut gözlemleri ağırlıklandırıp yeni veri noktaları üretmek
- Buna ek olarak, çoğunluk sınıfını bir miktar küçültmek için  
  <a href="https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/">**`RandomUnderSampler`**</a> da deneyebilirsiniz

In [3]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)


X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42
)

smote = SMOTE(sampling_strategy=0.5, random_state=42)

X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)


scaler = StandardScaler()


X_train_scaled = scaler.fit_transform(X_train_bal)


X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

log_reg = LogisticRegression(
    max_iter=1000,
    random_state=42
)

log_reg.fit(X_train_scaled, y_train_bal)
y_val_pred_lr = log_reg.predict(X_val_scaled)
y_val_proba_lr = log_reg.predict_proba(X_val_scaled)[:, 1]

print(classification_report(y_val, y_val_pred_lr))
print("Val ROC-AUC:", roc_auc_score(y_val, y_val_proba_lr))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     42647
           1       0.22      0.85      0.34        74

    accuracy                           0.99     42721
   macro avg       0.61      0.92      0.67     42721
weighted avg       1.00      0.99      1.00     42721

Val ROC-AUC: 0.9665769716066336


## 2. Sinir Ağı yinelemeleri

Sınıflarınızı yeniden dengelediğinize göre, test puanınızı optimize etmek için bir sinir ağı uyarlayın. Aşağıdaki ipuçlarını kullanmaktan çekinmeyin:

- Girişlerinizi normalleştirin!
    - Modelinizdeki ön işlemeyi “boru hattı” haline getirmek için tercihen model içinde bir [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) katmanı kullanın. 
    - Veya modelinizin dışında sklearn'in [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) öğesini kullanın, `X_train`, `X_val` ve `X_test` öğelerini uygulayın.
- Modelinizi aşırı uyumlu hale getirin, ardından aşağıdakileri kullanarak düzenleyin:
- Erken Durdurma kriterleri
- [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) katmanları
    - veya [`düzenleyiciler`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) katmanları
- 🚨 İzlemek istediğiniz metrikleri ve kullanmak istediğiniz kayıp fonksiyonunu dikkatlice düşünün!

In [4]:
overfit_model = models.Sequential([
    layers.Dense(512, activation="relu", input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

overfit_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC(name="auc")
    ]
)

history_overfit = overfit_model.fit(
    X_train_scaled, y_train_bal,
    validation_data=(X_val_scaled, y_val),
    epochs=10,
    batch_size=256,
    verbose=1
)


/home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - auc: 0.9996 - loss: 0.0179 - precision: 0.9937 - recall: 0.9893 - val_auc: 0.9482 - val_loss: 0.0358 - val_precision: 0.1713 - val_recall: 0.8378
Epoch 2/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 16s 14ms/step - auc: 0.9998 - loss: 0.0050 - precision: 0.9975 - recall: 0.9984 - val_auc: 0.9178 - val_loss: 0.0105 - val_precision: 0.4833 - val_recall: 0.7838
Epoch 3/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - auc: 0.9999 - loss: 0.0030 - precision: 0.9985 - recall: 0.9992 - val_auc: 0.9155 - val_loss: 0.0272 - val_precision: 0.2105 - val_recall: 0.8108
Epoch 4/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - auc: 0.9999 - loss: 0.0030 - precision: 0.9985 - recall: 0.9992 - val_auc: 0.9035 - val_loss: 0.0139 - val_precision: 0.3494 - val_recall: 0.7838
Epoch 5/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - auc: 0.9999 - loss: 0.0025 - precision: 0.9987 - recall: 0.9993 - val_auc: 0.9049 - val_loss: 0.0104 - val_precision: 0.6020 -

In [5]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    mode="max",
    patience=5,
    restore_best_weights=True
)
best_model = models.Sequential([
    layers.Dense(
        256,
        activation="relu",
        kernel_regularizer=regularizers.l2(1e-4),
        input_shape=(X_train_scaled.shape[1],)
    ),
    layers.Dropout(0.5),

    layers.Dense(
        128,
        activation="relu",
        kernel_regularizer=regularizers.l2(1e-4)
    ),
    layers.Dropout(0.3),

    layers.Dense(1, activation="sigmoid")
])
best_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC(name="auc")
    ]
)
history_best = best_model.fit(
    X_train_scaled, y_train_bal,
    validation_data=(X_val_scaled, y_val),
    epochs=10,
    batch_size=256,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - auc: 0.9980 - loss: 0.0654 - precision_1: 0.9869 - recall_1: 0.9648 - val_auc: 0.9473 - val_loss: 0.0305 - val_precision_1: 0.3605 - val_recall_1: 0.8378
Epoch 2/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - auc: 0.9996 - loss: 0.0294 - precision_1: 0.9926 - recall_1: 0.9909 - val_auc: 0.9415 - val_loss: 0.0230 - val_precision_1: 0.4207 - val_recall_1: 0.8243
Epoch 3/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - auc: 0.9998 - loss: 0.0216 - precision_1: 0.9943 - recall_1: 0.9947 - val_auc: 0.9288 - val_loss: 0.0220 - val_precision_1: 0.3588 - val_recall_1: 0.8243
Epoch 4/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - auc: 0.9998 - loss: 0.0185 - precision_1: 0.9951 - recall_1: 0.9961 - val_auc: 0.9303 - val_loss: 0.0186 - val_precision_1: 0.4729 - val_recall_1: 0.8243
Epoch 5/10
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - auc: 0.9998 - loss: 0.0170 - precision_1: 0.9955 - recall_1: 0.9966 - val_auc: 0.9242 - val_lo

In [6]:
print("Neural Network Test Results:")
best_model.evaluate(X_test_scaled, y_test, verbose=0)

y_test_proba_nn = best_model.predict(X_test_scaled).ravel()
print("Test ROC-AUC:", roc_auc_score(y_test, y_test_proba_nn))


Neural Network Test Results:
1336/1336 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Test ROC-AUC: 0.975858948425071


### 🧪 Kodunu Test Et

Orijinal dengesiz veri kümesinin (`X_test`, `y_test`) temsil edici bir örneğinde gerçek test performansınızın altında kalan sonuçları `precision` ve `recall` değişkenlerine kaydedin.

In [12]:
X_test_rep, _, y_test_rep, _ = train_test_split(
    X_test_scaled,
    y_test,
    test_size=0.7,          
    stratify=y_test,
    random_state=42
)

y_test_rep_proba = best_model.predict(X_test_rep).ravel()
y_test_rep_pred = (y_test_rep_proba >= 0.5).astype(int)


precision = precision_score(y_test_rep, y_test_rep_pred)
recall = recall_score(y_test_rep, y_test_rep_pred)
print(f"Test Precision : {precision:.2f}")
print(f"Test Recall : {recall:.2f}")


401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Test Precision : 0.40
Test Recall : 0.82


In [13]:
from nbresult import ChallengeResult

result = ChallengeResult('solution',
    precision=precision,
    recall=recall,
    fraud_number=len(y_test[y_test == 1]),
    non_fraud_number=len(y_test[y_test == 0]),
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S18D3-S-Data-credit-card-challenge/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 2 items

test_solution.py::TestSolution::test_is_score_good_enough PASSED         [ 50%]
test_solution.py::TestSolution::test_is_test_set_representative PASSED   [100%]

============================== 2 passed in 0.03s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## 🏁 İsteğe bağlı: Bu zorluk için Google'ın çözümünü okuyun
Bu oturumdaki tüm zorlukları tamamladığınız için tebrikler!

Son olarak, Google'ın kendi çözümünü doğrudan [Colab'da buradan](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb) okuyun. 

İlginç teknikler ve en iyi uygulamaları keşfedeceksiniz.